# Rebinning Multi-Analyzer data from ID22 taken with a 2D detector

This notebooks presents how to agregate together on a same $2\theta$ grid all data coming from all pixels of the Eiger 2M detector.

## Introduction of the Multi-Analyzer stage for high resolution powder diffraction:

![eiger](multianalyzer3.png)
![multianalyzer](multianalyzer2.jpeg)

Please ensure you first read: [J. Appl. Cryst. (2021). 54, 1088-1099](https://doi.org/10.1107/S1600576721005288).

Author: Jérôme Kieffer, ESRF

In [1]:
%matplotlib nbagg
# Compile the library
! cd .. && python3 setup.py build && cd -

INFO:root:Generating grammar tables from /usr/lib/python3.9/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.9/lib2to3/PatternGrammar.txt
INFO:multianalyzer.setup:Use setuptools with cython
INFO:multianalyzer.setup:Install requires: numpy >=1.19.5
running build
running build_py
running build_ext
/users/kieffer/workspace-400/multianalyzer/sandbox


In [2]:
import sys
import os
import time
import numpy
import h5py
import json
from matplotlib.pyplot import subplots
from matplotlib import colors
from scipy.signal import find_peaks
sys.path.append("../build/lib.linux-x86_64-3.9/")
from multianalyzer import MultiAnalyzer, ID22_bliss_parser, topas_parser
start_time = time.perf_counter()

In [ ]:
%%time
#Nota this cell takes a while to read all frames and perform a max-filter on the stack
!pyFAI-average -m max -F numpy -o max.npy /mnt/data/ID22/MultiAnalyzer/LaB6_31keV_150921/LaB6_31keV_150921_0001/LaB6_31keV_150921_0001.h5::/1.1/instrument/eiger/data

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [ ]:
fig, ax = subplots(figsize=(10,5))
img = numpy.load("max.npy")
ax.imshow(img, norm=colors.LogNorm(vmin=1, vmax=65000))

## Read the configuration of the various analyzers

In [ ]:
topas = "out7.pars"
print(open(topas).read())

In [ ]:
param = topas_parser(topas)
print(json.dumps(param, indent=2))

## Read the ROI-collection from HDF5 file. 

This is a substential amount of data and takes up to a minute!

In [ ]:
%%time 
hdf5 = "/mnt/data/ID22/MultiAnalyzer/LaB6_35keV_mantr10_nozzle20_1/id222108_LaB6_35keV_mantr10_nozzle20_1.h5"
hdf5_data = ID22_bliss_parser(hdf5)

In [ ]:
print(hdf5_data)

In [ ]:
print("Shape of the ROI-collection:", hdf5_data["roicol"].shape)

## Perform  the rebinning

In [ ]:
# Ensure all units are consitent. Here lengths are in milimeters.
L = param["L1"]
L2 = param["L2"]
pixel = 75e-3

# Angles are all given in degrees
center = numpy.array(param["centre"])
psi = numpy.rad2deg(param["offset"])
rollx = numpy.rad2deg(param["rollx"])
rolly = numpy.rad2deg(param["rolly"])

tha = hdf5_data["tha"]
thd = hdf5_data["thd"]

# Finally initialize the rebinning engine.
mma = MultiAnalyzer(L, L2, pixel, center, tha, thd, psi, rollx, rolly)

In [ ]:
#Display the 2theta position of all ROI for a given arm position
arm = 10
fig, ax = subplots()
x = range(512)
for a in range(13):
    ax.plot(x, [mma.refine(i, a, arm, phi_max=8) for i in x], label=f"MA{a}")

ax.set_xlabel("Pixel index")
ax.set_xlabel(r"$2\theta$ position of pixel")
ax.set_title(f"Arm is at {arm} °")
ax.legend()

In [ ]:
%%time 
roicol = hdf5_data["roicol"]
arm = hdf5_data["arm"]
mon = hdf5_data["mon"]
res = mma.integrate(roicol, arm, mon, 3.383800, 45.099600, 7e-4, 10)

In [ ]:
#Load the reference data:
ref = numpy.loadtxt("LaB6_35keV_all.xye").T

In [ ]:
fig, ax = subplots()
scale = ref[1].max() / numpy.nanmax(res[1]/res[2])
for i in range(0, 13, 1):
    ax.plot(res[0], scale*res[1][i]/res[2][i], "--", label=f"MA{i}")
ax.plot(ref[0],ref[1], "-b", label="ref")
ax.set_xlabel(r"$2\theta$ (°)")
ax.set_ylabel("Scattered intensity")
ax.set_title("LaB6")
ax.legend()

## Perform some basic profile analysis

This is not a Rietveld, nor a Pawley refinement but it gives some insights on the intrinsic width of the signal

In [ ]:
pk = find_peaks(ref[1], width=(5, 50), prominence=1000)

In [ ]:
fig,ax = subplots()
ax.plot(ref[1])
ax.plot(pk[0], pk[1]["prominences"], ".")
ax.set_xlabel("index")

In [ ]:
fig, ax = subplots()
delta = (ref[0][-1] - ref[0][0])/(len(ref[0])-1)
ax.set_xlabel(r"$2\theta$ (°)")
ax.set_ylabel("FWHM (°)")
for i in range(0, 13, 1):
    sig = scale*res[1][i, :-1]/res[2][i, :-1]
    pks = find_peaks(sig, width=(5, 50), prominence=1000)
    ax.plot(res[0][:-1][pks[0]], delta*pks[1]["widths"], "--", label=f"MA{i}")
ax.plot(ref[0][pk[0]], delta*pk[1]["widths"], "-b", label="Ref")
ax.legend()


## Save data as text file

In [ ]:
res2 = numpy.vstack((res[0], scale*res[1]/res[2]))
numpy.savetxt("LaB6.xyn", res2.T, fmt='%.4f')

## Conclusion
This notebook explains how to rebin data coming from the new experimental setup from ID22.

In [ ]:
print(f"Total execution time: {time.perf_counter()-start_time:.3f}s")